In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from progressbar import progressbar as p
import spacy
warnings.filterwarnings('ignore')

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
df=pd.read_csv('train.csv')
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [17]:
ques=list(df['question1'])+list(df['question2'])
tfidf=TfidfVectorizer(lowercase=False)
tfidf.fit_transform(ques)

wordtf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [19]:
nlp = spacy.load('en_core_web_sm')
ls=[list(df['question1']),list(df['question2'])]

for i,l in enumerate(ls):
    vecs=[]
    for q in p(l):
        
        doc=nlp(q)
        mean_vec=np.zeros([len(doc),len(doc[0].vector)])
        for word in doc:
            vec=word.vector
            
            try:
                idf = wordtf[str(word)]
                
            except:
                idf = 0
        
            mean_vec += vec * idf
        mean_vec = mean_vec.mean(axis=0)
        vecs.append(mean_vec)
    df['w2v_q_'+str(i)]=list(vecs)
    
    
    
        


100% (404290 of 404290) |################| Elapsed Time: 0:56:47 Time:  0:56:47
100% (404290 of 404290) |################| Elapsed Time: 1:04:48 Time:  1:04:48


In [ ]:
df.to_csv('w2v.csv')

In [39]:
#Reading Processed files

nlp=pd.read_csv('nlp_features.csv',encoding='latin-1')
pre_nlp=pd.read_csv('pre_nlp.csv',encoding='latin-1')

In [ ]:
#Preprocessing dataframes for final table
nlp=nlp.drop(nlp.columns.tolist()[1:5],axis=1)
pre_nlp=pre_nlp.drop(pre_nlp.columns.tolist()[1:6],axis=1)
df=df.drop(df.columns.tolist()[1:6],axis=1)

In [45]:
#Converting list to Dataframe
dfq1=pd.DataFrame(df['w2v_q_0'].values.tolist(),index=df.index)
dfq2=pd.DataFrame(df['w2v_q_1'].values.tolist(),index=df.index)

In [ ]:
#merging all and saving
if not os.path.isfile('final_features.csv'):
    dfq1['id']=nlp['id']
    dfq2['id']=nlp['id']
    nlp  = nlp.merge(pre_nlp, on='id',how='left') #nlp=nlp+pre
    pre_nlp  = dfq1.merge(dfq2, on='id',how='left') #pre=q1+q2
    result  = nlp.merge(pre_nlp, on='id',how='left') #rsult=pre+nlp
    result.to_csv('final_features.csv')
